In [8]:
import torch
import yaml

from data import MainDataset
from model import MixtureAttention
from utils import DotDict

In [9]:
config_file_path = '../configs/attn_lstm_vocab_50k.yml'
with open(config_file_path, 'r') as f:
    config = DotDict(yaml.safe_load(f))

In [15]:
data_train = MainDataset(
    N_filename = '../' + config.data.N_filename,
    T_filename = '../' + config.data.T_filename,
    is_train=True,
    truncate_size=config.data.truncate_size
)

reading data from  .././pickle_data/PY_non_terminal_small.pickle
the vocab_sizeN is 13 (not including the eof)
the number of training data is 14
the number of test data is 14

reading data from  .././pickle_data/PY_terminal_50k_whole.pickle
the vocab_sizeT is 50000 (not including the unk and eof)
the attn_size is 50
the number of training data is 14
the number of test data is 14
Finish reading data and take 0.02



100%|██████████| 14/14 [00:00<00:00, 2012.07it/s]


In [16]:
device = config.train.device

if not torch.cuda.is_available():
    device = 'cpu'

In [17]:
model = MixtureAttention(
        hidden_size = config.model.hidden_size,
        vocab_sizeT = data_train.vocab_sizeT,
        vocab_sizeN = data_train.vocab_sizeN,
        attn_size = data_train.attn_size,
        embedding_sizeT = config.model.embedding_sizeT,
        embedding_sizeN = config.model.embedding_sizeN,
        num_layers = config.model.num_layers,
        dropout = config.model.dropout,
        label_smoothing = config.model.label_smoothing,
        pointer = config.model.pointer,
        attn = config.model.attn,
        device = device
    )

In [18]:
checkpoint = torch.load('../checkpoints/attn_lstm_vocab_50k/epoch_0000.pth')

In [19]:
model.load_state_dict(checkpoint['model'])

<All keys matched successfully>

In [22]:
from torch import Tensor

p_tensor = Tensor([0, 1, 2])
t_tensor = Tensor([0, 1, 2])
n_tensor = Tensor([0, 1, 2])

In [52]:
example = data_train.data[0]
#t = Tensor([1,3], type=torch.long, device=device),
t = torch.LongTensor([example[0]], device=device)

below predictions using some train data are being made

In [56]:
from torch import LongTensor

example = data_train.data[3]

with torch.no_grad():
    prediction = model(
        LongTensor([example[0]], device=device),
        LongTensor([example[1]], device=device),
        LongTensor([example[2]], device=device)
    )